# Introduction/ Business Problem

A restaurant's location is one the most important factors for its success. So, it is critical to find the best location for opening restaurant.

There is the task to establish a Chinese restaurant in a Toronto neighborhood. So the questions is - which neighborhood will be the optimal?

# Data

Following data will be used for assessment of each neighborhood:

1. Neighbourhood names, alongside their corresponding boroughs and postal codes, scraped from Wikipedia: (https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M). We will obtain the list of neighborhoods of Toronto.
2. Geographical coordinates of each neighbourhood: (https://cocl.us/Geospatial_data).
3. The Foursquare API will be used to explore neighborhood venues in Toronto. We will check presence of Chinese restaurants in each neighborhood to check possible competition.
4. The Toronto's census data for its social demographic characteristics will be distilled from Toronto's Neighborhood Profile (https://bit.ly/3airrOJ). We will obtain spending power and distribution of ethnic groups.